In [3]:
#!pip3 install -U botocore

In [2]:
import boto3
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = 'reinvent-cola-model-artifacts'#sagemaker_session.default_bucket()
prefix = 'roberta/cola'

role = sagemaker.get_execution_role()

In [2]:
import os
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = local


In [3]:
from source.preproc import *
import argparse
#import logging
from tqdm import tqdm, trange, tqdm_notebook
import numpy as np
import io
from transformers import (BertConfig,
                          BertForSequenceClassification,
                          BertTokenizer)
from torch.utils.data import (TensorDataset,
                              DataLoader,
                              RandomSampler,
                              SequentialSampler)

#logger = logging.getLogger(__name__)
#logger.setLevel(logging.DEBUG)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPERPARAMS
batch_size = 32

def transer_gpu_dataloader(X, mask, y, batch_size, device):
    X = X.to(device)
    mask = mask.to(device)
    y = y.to(device)
    data = TensorDataset(X, mask, y)
    sampler = RandomSampler(data)
    return DataLoader(data, sampler=sampler, batch_size=batch_size)

In [ ]:
%%bash

https://nyu-mll.github.io/CoLA/
wget https://nyu-mll.github.io/CoLA/cola_public_1.1.zip
unzip cola_public_1.1.zip

In [4]:
def read_data(dset='in_domain_train'):
    fname = './cola_public/raw/{}.tsv'.format(dset)
    df = pd.read_csv(fname, delimiter='\t', header=None, 
                    names=['sentence_source', 'label', 'notes', 'sentence'])
    return df

def prepare_dataset(dset, tokenizer, max_len):
    df = read_data(dset)
    sentences, labels = pre_process(df)
    tokenized_texts, input_ids = tokenize_and_ids(sentences, tokenizer)
    if dset=='train':
        plot_token_dist(tokenized_texts)
    padded_input_ids, attention_masks = pad_sequence_mask(input_ids, max_len)
    return padded_input_ids, attention_masks, labels

tokenizer = define_tokenizer()
max_len = 64 #128
X_train, mask_train, y_train = prepare_dataset('in_domain_train', tokenizer, max_len)
X_valid, mask_valid, y_valid = prepare_dataset('in_domain_dev', tokenizer, max_len)

Total vocab size:  30522
Pad token ID:  0


In [5]:
train_dataloader = transer_gpu_dataloader(X_train, mask_train, y_train, batch_size, device)
valid_dataloader = transer_gpu_dataloader(X_valid, mask_valid, y_valid, batch_size, device)

# Data Preview

In [3]:
# get some random training images
dataiter = iter(train_dataloader)
text, _,  labels = dataiter.next()

# show images
print(text)

# print labels
print(' '.join('%9s' % labels[j] for j in range(4)))

NameError: name 'train_dataloader' is not defined

In [7]:
inputs = sagemaker_session.upload_data(path='./cola_public/raw/', bucket=bucket, key_prefix='data/cola')

In [8]:
from sagemaker.pytorch import PyTorch

instance_type = 'ml.p3.2xlarge'

hyperparameters = {'max_len': 64,
                          'batch_size': 64,
                          'gradient_accumulation_steps': 1,
                          'num_train_epochs': 1,
                          'lr': 5e-5,
                          'eps': 1e-8}

estimator = PyTorch(entry_point='train.py',
                            source_dir='source',
                            role=role,
                            framework_version='1.1.0',
                            train_instance_count=1,
                            train_instance_type=instance_type,
                            hyperparameters=hyperparameters)

In [9]:
estimator.fit(inputs)

2019-11-27 16:05:20 Starting - Starting the training job...
2019-11-27 16:05:22 Starting - Launching requested ML instances...
2019-11-27 16:06:18 Starting - Preparing the instances for training......
2019-11-27 16:07:20 Downloading - Downloading input data...
2019-11-27 16:07:36 Training - Downloading the training image...
2019-11-27 16:08:13 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-11-27 16:08:14,369 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-11-27 16:08:14,393 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-11-27 16:08:15,825 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-11-27 16:08:16,110 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-11-27 16:08:16,110 sagemaker-c

In [11]:
#!aws s3 rm s3://sagemaker-ap-southeast-1-232392939564/ --recursive

# Deploy the trained model to prepare for predictions 

In [11]:
from sagemaker.pytorch import PyTorchModel

instance_type = 'ml.p2.8xlarge'

predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

Using already existing model: sagemaker-pytorch-2019-11-27-16-05-20-009


--------------------------------------------------------------------------------------------------------------!

In [1]:
preds = predictor.predict(text.cpu().numpy())

assert np.argmax(preds[0].cpu().numpy(),-1).shape[0] ==  labels.shape[0]

NameError: name 'predictor' is not defined